In [1]:
import torchvision
import torch
import PIL as Image

In [2]:
# defining constants
NUM_OF_CLASSES = 10
BATCH_SIZE = 32
NUM_OF_EPOCHS = 10
# initial transformation
resize_transformation = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))
])
# downloading and loading the dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=resize_transformation)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=resize_transformation)

100%|██████████| 9912422/9912422 [00:10<00:00, 968595.45it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 1978264.32it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [3]:
# loading the dataset into the data loader
train_data = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False)

In [4]:
# defining the LeNet CNN architecture
class LeNet5(torch.nn.Module):
    def __init__(self, num_of_classes):
        super().__init__()
        # CNN part - feature extraction
        self.feature_extraction = torch.nn.Sequential(
            # convolutional layer 1
            torch.nn.Conv2d(in_channels = 1, kernel_size = (5, 5), padding=0, stride=1, out_channels = 6, bias = True),
            torch.nn.Tanh(), 
            torch.nn.MaxPool2d(kernel_size = (2, 2)),
            # convolutional layer 2
            torch.nn.Conv2d(in_channels = 6, kernel_size = (5, 5), padding=0, stride=1, out_channels = 16, bias = True), 
            torch.nn.Tanh(),
            torch.nn.MaxPool2d(kernel_size = (2,2))
        )
        # classification part - Multilayer perceptron region
        self.classifier = torch.nn.Sequential(
            # classifier - Multi layer perceptron
            # layer 1
            torch.nn.Linear(in_features = 400, out_features = 120),
            torch.nn.ReLU(), 
            # layer 2
            torch.nn.Linear(in_features = 120, out_features = 84),
            torch.nn.ReLU(), 
            # layer 3
            torch.nn.Linear(in_features = 84, out_features = 10), 
            torch.nn.Softmax()
        )
    # forward propogation of the network
    def forward(self, X):
        X = self.feature_extraction(X)
        X = torch.flatten(X, 1)   
        X = self.classifier(X)
        return X
# defining the model 
model = LeNet5(NUM_OF_CLASSES)

In [5]:
# setting the optimizer for the model 
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [6]:
# Training loop for the LeNet CNN architecture          
for epoch in range(NUM_OF_EPOCHS):
    for batch_index, (data, label) in enumerate(train_data):
        logits = model(data)
        cost = torch.nn.functional.cross_entropy(logits, label)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        # logging
    #     if (batch_index%50==0):
    #         print(f"Epoch : {epoch+1} | Batches : {batch_index}/{len(train_data)} | Loss Value : {cost.item()}")
    # print(f"Epoch : {epoch} | Loss Value : {cost.item()}")

c:\Users\Public\ipenv\envs\learndl\lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [7]:
# computing the training and validation loss and accuracy
with torch.no_grad():
    correct_pred = 0
    num_examples = 0
    for i, (features, labels) in enumerate(train_data):
        logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        correct_pred += torch.sum(predictions == labels).item()
        num_examples += labels.size(0)
    train_acc = (correct_pred / num_examples) * 100
    correct_pred = 0
    num_examples = 0
    for i, (features, labels) in enumerate(test_data):
        logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        correct_pred += torch.sum(predictions == labels).item()
        num_examples += labels.size(0)
    valid_acc = (correct_pred / num_examples) * 100

In [8]:
print(f"Training accuracy of the LeNet5 model: {train_acc:.2f}%")
print(f"Test accuracy of the LeNet5 model: {valid_acc:.2f}%")

Training accuracy of the LeNet5 model: 92.78%
Test accuracy of the LeNet5 model: 92.93%
